

---
# 난독화 복원 솔루션: 더블 모델



In [1]:
# =============================================================================
# 0. 환경 설정
# =============================================================================
!pip install transformers

from google.colab import drive
drive.mount('/content/drive')

import time
import re
import torch
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# =============================================================================
# 1. 경로 및 프롬프트 설정
# =============================================================================
# [Signal] 분류기
PATH_CLASSIFIER = "/content/drive/MyDrive/P01_Nandoc/Classifier"

# [Core 1] 오타 담당 (팀원이 학습시킨 프롬프트 적용 모델)
PATH_MODEL_TYPO = "/content/drive/MyDrive/P01_Nandoc/Finalmodel1_3"

# [Core 2] 야민정음 담당
PATH_MODEL_YAMIN = "/content/drive/MyDrive/P01_Nandoc/dict+jamoB+kobart_ft+cleanData/"

# 🔥 [중요] 오타 모델용 프롬프트 (학습 코드에서 발췌)
KOR_PROMPT = (
    "아래 입력은 난독화된 한국어 문장을 자모(초성/중성/종성)로 분해한 문자열이야.\n"
    "이 자모들을 한국어 문장 그대로 복원해.\n"
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Inference Device: {device}")

# =============================================================================
# 2. 통합 복원 시스템 클래스
# =============================================================================
class NandocSystem:
    def __init__(self):
        print("🔄 [System] 엔진 시동 중... (Prompt-Aware Mode)")
        start_time = time.time()

        # A. 분류기 로드
        try:
            self.cls_tokenizer = ElectraTokenizer.from_pretrained(PATH_CLASSIFIER)
            self.cls_model = ElectraForSequenceClassification.from_pretrained(PATH_CLASSIFIER).to(device)
            self.cls_model.eval()
            print("✅ [1/3] 분류기(Classifier) 장착 완료")
        except Exception as e:
            print(f"❌ 분류기 로드 실패: {e}")
            raise

        # B. 오타 모델 로드 (Prompt 적용 모델)
        try:
            self.typo_tokenizer = PreTrainedTokenizerFast.from_pretrained(PATH_MODEL_TYPO)
            self.typo_model = BartForConditionalGeneration.from_pretrained(PATH_MODEL_TYPO).to(device)
            self.typo_model.eval()
            print("✅ [2/3] 오타 모델(Typo Core) 장착 완료")
        except Exception as e:
            print(f"❌ 오타 모델 로드 실패: {e}")
            raise

        # C. 야민 모델 로드
        try:
            self.yamin_tokenizer = PreTrainedTokenizerFast.from_pretrained(PATH_MODEL_YAMIN)
            self.yamin_model = BartForConditionalGeneration.from_pretrained(PATH_MODEL_YAMIN).to(device)
            self.yamin_model.eval()
            print("✅ [3/3] 야민 모델(Yamin Core) 장착 완료")
        except Exception as e:
            print(f"❌ 야민 모델 로드 실패: {e}")
            raise

        print(f"🎉 [System] 시스템 준비 완료 ({time.time() - start_time:.2f}s)\n")

    # -------------------------------------------------------------------------
    # 🛡️ 방어 로직 1: 이중 잠금 (Double Lock) -> 오타 모델용 (강력함)
    # -------------------------------------------------------------------------
    def _double_lock_cut(self, input_text, pred_text):
        # 1. 문장 개수 동기화
        input_punctuations = len(re.findall(r'[.!?]', input_text))
        target_count = max(input_punctuations, 1)

        matches = [m.start() for m in re.finditer(r'[.!?]', pred_text)]
        if len(matches) >= target_count:
            cut_idx = matches[target_count - 1]
            candidate = pred_text[:cut_idx+1]
        else:
            candidate = pred_text

        # 2. 길이 제한 (1.3배)
        limit_len = int(len(input_text) * 1.3) + 10
        if len(candidate) > limit_len:
            truncated = candidate[:limit_len]
            # 안전하게 자르기
            last_punc = max(truncated.rfind('.'), truncated.rfind('!'), truncated.rfind('?'))
            if last_punc != -1: return truncated[:last_punc+1]
            return truncated
        return candidate

    # -------------------------------------------------------------------------
    # 🛡️ 방어 로직 2: 단순 정제 (Simple Polish) -> 야민 모델용 (유연함)
    # -------------------------------------------------------------------------
    def _simple_defense(self, text):
        # 1. 특수문자 청소 (기본적인 것만)
        clean_pattern = r'[^가-힣a-zA-Z0-9\s.,?!\'"%~]'
        text = re.sub(clean_pattern, '', text)

        # 2. 다중 공백 제거
        text = re.sub(r'\s+', ' ', text).strip()

        # 3. 중복 문장 제거 (간단 버전)
        sentences = re.split(r'(?<=[.?!])\s+', text)
        unique_sentences = []
        for s in sentences:
            s = s.strip()
            if not s: continue
            if unique_sentences and s == unique_sentences[-1]:
                continue
            unique_sentences.append(s)
        return ' '.join(unique_sentences)

    # -------------------------------------------------------------------------
    # 🚀 메인 추론 함수 (Restore)
    # -------------------------------------------------------------------------
    def restore(self, text):
        if not text: return ""

        # Step 1. 분류 (Classifier)
        cls_inputs = self.cls_tokenizer(
            text, return_tensors='pt', truncation=True, max_length=128
        ).to(device)

        with torch.no_grad():
            outputs = self.cls_model(**cls_inputs)
            # 0: 오타/일반, 1: 야민정음
            class_id = torch.argmax(outputs.logits, dim=-1).item()
            probs = torch.softmax(outputs.logits, dim=-1)[0]
            confidence = probs[class_id].item()

        # Step 2. 라우팅 & 생성 (Routing & Generation)

        # ====================================================
        # [Case A] 오타 모델 (TYPE-0)
        # 특징: 프롬프트 필요, 강력한 방어 로직 적용
        # ====================================================
        if class_id == 0:
            target_model = self.typo_model
            target_tokenizer = self.typo_tokenizer
            info = "TYPE-0 (오타)"

            # 🔥 [핵심] 프롬프트 결합
            final_input = KOR_PROMPT + text

            gen_inputs = target_tokenizer(
                final_input, return_tensors='pt', padding=True, truncation=True, max_length=512 # 프롬프트 때문에 길이 넉넉히
            ).to(device)

            with torch.no_grad():
                gen_ids = target_model.generate(
                    gen_inputs['input_ids'],
                    attention_mask=gen_inputs['attention_mask'],
                    num_beams=5,
                    no_repeat_ngram_size=3,
                    repetition_penalty=1.5,
                    early_stopping=True,
                    max_length=256
                )
            raw_output = target_tokenizer.decode(gen_ids[0], skip_special_tokens=True)

            # 🔥 [핵심] 강력한 방어 (Double Lock)
            processed_text = self._double_lock_cut(text, raw_output)
            final_text = self._simple_defense(processed_text) # 마무리 청소

        # ====================================================
        # [Case B] 야민 모델 (TYPE-1)
        # 특징: 프롬프트 없음, 유연한 방어 로직 적용
        # ====================================================
        else:
            target_model = self.yamin_model
            target_tokenizer = self.yamin_tokenizer
            info = "TYPE-1 (야민)"

            # 원본 그대로 입력
            final_input = text

            gen_inputs = target_tokenizer(
                final_input, return_tensors='pt', padding=True, truncation=True, max_length=128
            ).to(device)

            with torch.no_grad():
                gen_ids = target_model.generate(
                    gen_inputs['input_ids'],
                    attention_mask=gen_inputs['attention_mask'],
                    num_beams=5,
                    no_repeat_ngram_size=3,
                    repetition_penalty=1.5,
                    early_stopping=True,
                    max_length=256
                )
            raw_output = target_tokenizer.decode(gen_ids[0], skip_special_tokens=True)

            # 🔥 [핵심] 간단한 방어 (Simple Defense)
            final_text = self._simple_defense(raw_output)

        return final_text, info, confidence


Mounted at /content/drive
🚀 Inference Device: cuda


In [4]:

# =============================================================================
# 3. 실행 테스트
# =============================================================================
if __name__ == "__main__":
    nandoc = NandocSystem()

    print("\n" + "="*60)
    print("💡 [Nandoc Double-Engine] 가동 (Prompt: ON for Typo)")
    print("="*60)

    while True:
        user_input = input("\n📝 입력: ")
        if user_input.lower() in ['q', 'quit']:
            print("👋 종료합니다.")
            break

        if not user_input.strip(): continue

        result, type_info, conf = nandoc.restore(user_input)

        print(f"   └─ [{type_info} | {conf:.0%}] -> {result}")


🔄 [System] 엔진 시동 중... (Prompt-Aware Mode)
✅ [1/3] 분류기(Classifier) 장착 완료
❌ 오타 모델 로드 실패: Can't load tokenizer for '/content/drive/MyDrive/P01_Nandoc/Finalmodel1_3'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/content/drive/MyDrive/P01_Nandoc/Finalmodel1_3' is the correct path to a directory containing all relevant files for a PreTrainedTokenizerFast tokenizer.


OSError: Can't load tokenizer for '/content/drive/MyDrive/P01_Nandoc/Finalmodel1_3'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/content/drive/MyDrive/P01_Nandoc/Finalmodel1_3' is the correct path to a directory containing all relevant files for a PreTrainedTokenizerFast tokenizer.

In [ ]:
# =============================================================================
# 3. 5개 문장 자동 테스트 실행
# =============================================================================
if __name__ == "__main__":
    # 1. 시스템 초기화
    nandoc = NandocSystem()

    # 2. 테스트 문장 리스트 (오타, 야민정음, 혼합)
    test_sentences = [
        "아... 까껵 초코 쀼됴 뻥 뜛려셔 시원하자만 담배 냄쌢 밁버림. 싸게 하루만 묵껬댜! 하는 샤람한테만 츄천. 땸빼 냄새까 모튼 짱졈을 카쳝갸는 꼿. 놃래빵에써 캭종 댬빼와 유흐에 쪌였을 때 나는 냄새까 곘쑉 뱌에 이씀 ㅆ... 싸니까 할 먈 없음.",
        "찌안 츅쳐늚료 따너왔는떼 꺄쎵빠 쬬교 깔끔햐네요. 쩌렴햔뗖 물콰 음료똦 았꾜 빵뚀 아쮼 땨뜻했슴나땾. 휘낙쓰밙그와 껴러뚀 멸저 않얆서 쬬햤꾦용. 뺘롤 엿베 펴늬쩌멃 어쎠쎠 라했씀나땨. 땮으메또 아옹할 쓬 악쓰면 엋용햐꾜 쌉네요. 옊아까 어렵찌먄 ^^",
        "호텔은 깔끔하고 서Bs도 좋았는데, bangOl 작Oㅏ 창문e 겨익 없어서 환71㉠r 안 되는 느낌0I 8었어yO. 그2Hㅌ Oㅏ침 싀ㅆ는 신선ㅎr그 맛있었으ni 만족^러웠습L|ㅁ-.",
        "01번 즉말O|| 친ヲ랑 go서 1박 2일했어요. bangOl 깔끔ㅎrヱ b㉰ 전망0l 경말 멋졌어8. 조싀E 간단Ha겨만 맛있ヱ, 직원분dl0l 친절hri서 분읶71E 좋았죠. ㅁ-만, 수영장 입7㉠r 조금 빽빽ㅎri서 조금 71㉰렸=데, 그 외O||는 완전 만족s러운 oㅕ행0I었어yO.",
        "펴난하 쨜 쓀꼬 왔씀낭땿. 쭈위에 먓쩝뚔 많아쎠 쬬흔 켯 꺄탸요."
    ]

    print("\n" + "="*80)
    print("📊 [Nandoc Intelligent System] 5개 문장 복원 테스트")
    print("="*80)

    for i, text in enumerate(test_sentences):
        # 복원 수행
        result, type_info, conf = nandoc.restore(text)

        print(f"\n[Test Case {i+1}]")
        print(f"🕵️‍♂️ 감지된 타입: {type_info} (확률: {conf:.2%})")
        print(f"📝 원본: {text[:60]}..." if len(text) > 60 else f"📝 원본: {text}")
        print(f"✨ 복원: {result}")
        print("-" * 80)
